In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os 
import numpy as np

GOLD_DATA_PATH = os.path.join("..", "..", "data/gold/")

In [ ]:
df4 = pd.read_csv(GOLD_DATA_PATH + "data_card_4_df.csv", sep=";", encoding = 'latin')

In [ ]:
df4 = df4.drop(columns=["Unnamed: 0"])
df4.set_index("Provincias", inplace=True)
df4.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df4['Producción media'] = scaler.fit_transform(df4['Producción media'].values.reshape(-1, 1))
df4['Porcentaje con dispositivo'] = scaler.fit_transform(df4['Porcentaje con dispositivo'].values.reshape(-1, 1))
df4['Mediana edad'] = scaler.fit_transform(df4['Mediana edad'].values.reshape(-1, 1))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df4_scaled = scaler.fit_transform(df4)
estimator = PCA(n_components=2)
X_pca = estimator.fit_transform(df4_scaled)
print(estimator.explained_variance_ratio_)
pd.DataFrame(np.matrix.transpose(estimator.components_), columns=['PC-1', 'PC-2'], index=df4.columns)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_pca[:,0], X_pca[:,1], s=50)

# anotación 
for i in range(0, len(X_pca)):
    ax.annotate(df4.iloc[i, :].name, (X_pca[i, 0], X_pca[i, 1]), fontsize=8)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

link_matrix_avg = linkage(df4, method='ward', metric='euclidean')
plt.figure(figsize=(8, 5))
dendrogram(link_matrix_avg, labels=df4.index)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

link_matrix = linkage(df4, method='single', metric='euclidean')
plt.figure(figsize=(8, 5))
dendrogram(link_matrix, labels=df4.index)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

link_matrix = linkage(df4, method='complete', metric='euclidean')
plt.figure(figsize=(8, 5))
dendrogram(link_matrix, labels=df4.index)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

link_matrix = linkage(df4, method='average', metric='euclidean')
plt.figure(figsize=(8, 5))
dendrogram(link_matrix, labels=df4.index)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import fcluster
import numpy as np

# Realizar el clustering jerárquico (usando la matriz de enlace creada previamente)
# 'linkage_matrix' debe estar calculada con el método 'ward' u otro método.
# # Paso 1: Generar muchos clusters inicialmente (granularidad alta)
# clusters_granular = fcluster(link_matrix_avg, t=10, criterion='maxclust')  # Generar 10 clusters

# # Agregar los clusters al DataFrame original para observar los resultados
# df4["Cluster_Granular"] = clusters_granular
# df4_reset = df4.reset_index()  # Reset the index to access "Provincias" column
# print("Clusters iniciales con granularidad alta:")
# print(df4_reset[["Provincias", "Cluster_Granular"]])

In [ ]:
# # Paso 2: Reagrupar o inspeccionar los clusters manualmente
# # Aquí observamos los tamaños de los clusters
# cluster_sizes = df4["Cluster_Granular"].value_counts()
# print("\nTamaño de cada cluster inicial:")
# print(cluster_sizes)

In [ ]:
# Paso 3: Cortar a un nivel lógico, por ejemplo, para obtener 5 clusters
clusters_final = fcluster(link_matrix_avg, t=4, criterion='maxclust')  # Cortar a 5 clusters

In [ ]:
# tratamos el cluster -1 como cluster de outliers
from sklearn import metrics
n_clusters_ = len(set(clusters_final)) - (1 if -1 in clusters_final else 0)
print('Estimated number of clusters: %d' % n_clusters_)
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(df4, clusters_final))

In [ ]:
#plotting orginal points with color related to label
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=clusters_final, s=50)
plt.legend(*scatter.legend_elements(), title="Clusters", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
df4["Cluster"] = clusters_final
res = df4[['Producción media', 'Porcentaje con dispositivo', 'Mediana edad', 'Cluster']].groupby('Cluster').mean()
res.plot(kind='bar', legend=True, xlabel='Cluster', ylabel='Media', title='Media de las variables por cluster')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
df4["Cluster"] = clusters_final
res = df4[['Familia de 1 padres y 0 hijos','Familia de 1 padres y 1 hijos','Familia de 1 padres y 2 hijos','Familia de 2 padres y 0 hijos','Familia de 2 padres y 1 hijos','Familia de 2 padres y 2 hijos', 'Cluster']].groupby('Cluster').mean()
res.plot(kind='bar', legend=True, xlabel='Cluster', ylabel='Media', title='Media de las variables por cluster')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
cluster_1 = df4[df4['Cluster'] == 1]
cluster_2 = df4[df4['Cluster'] == 2]
cluster_3 = df4[df4['Cluster'] == 3]
cluster_4 = df4[df4['Cluster'] == 4]

In [ ]:
cluster_1

In [ ]:
df4[["Cluster"]].to_csv("clusters_mapa.csv")